# Анализ игровых платформ и игр начала 2000-х

## Введение

### Цели и задачи проекта
- **Цель проекта** - провести анализ данных о видеоиграх, за период с 2000 по 2013, для выявления наиболее популярных игровых платформ и распределения игр по категориям в соответствии с полученными оценками от пользователей и критиков, с последующим подсчетом и анализом соотношений.
- **Задачи**:
- Ознакомиться с данными - изучить структуру и содержание таблицы, проверить корректность данных.
- Очистить данные - устранить дубликаты, обработать пропуски, привести данные к нужным типам.
- Ограничить период анализа - выделить данные только за 2000–2013 годы включительно.
- Категоризировать игры - присвоить каждой игре категорию отталкиваясь от оценки пользователя и критика (высокая, средняя, низкая).
- Определить лидирующие платформы - найти 7 платформ с наибольшим числом выпущенных игр за выбранный период.

### Описание данных
В проекте будут использованы данные датасета `new_games.csv` с таким описанием:
- `Name` — название игры.
- `Platform` — название платформы.
- `Year of Release` — год выпуска игры.
- `Genre` — жанр игры.
- `NA sales` — продажи в Северной Америке (в миллионах проданных копий).
- `EU sales` — продажи в Европе (в миллионах проданных копий).
- `JP sales` — продажи в Японии (в миллионах проданных копий).
- `Other sales` — продажи в других странах (в миллионах проданных копий).
- `Critic Score` — оценка критиков (от 0 до 100).
- `User Score` — оценка пользователей (от 0 до 10).
- `Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

### Содержимое проекта
1. Загрузка и знакомство с данными
2. Проверка ошибок в данных и их предобработка
    - Приведение наименований столбцов к единому стилю
    - Приведение типов данных к подходящему формату
    - Поиск и обработка пропусков
    - Поиск и обработка явных и неявных дубликатов
4. Фильтрация данных
5. Категоризация данных
   - Распределение игр по категориям
   - Топ-7 игровых платформ
6. Итоги проекта
   - Вывод
   - Проделанная работа

## Загрузка и знакомство с данными

In [1]:
# Импортируем библиотеку pandas.
import pandas as pd

In [2]:
# Выгружаем new_games.csv в датафрейм.
df = pd.read_csv('https:// ...

In [3]:
# Выводим первые пять строк для предварительной оценки данных.
df.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [4]:
# Вызываем метод info() для получения сводной информации о датафрейме (количество строк, столбцов, типы данных и количество непустых значений).
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [5]:
# Выводим количество пропусков в каждом столбце при помощи конструкции isna().sum().
df.isna().sum()

Name                  2
Platform              0
Year of Release     275
Genre                 2
NA sales              0
EU sales              0
JP sales              0
Other sales           0
Critic Score       8714
User Score         6804
Rating             6871
dtype: int64

Данные в целом соответствуют описанию по количеству столбцов и их наполнению, однако содержат особенности, которые могут затруднить дальнейший анализ.
К таким особенностям относятся:

- **Пропущенные значения в следующих столбцах:**
`Name` (**2**), `Year of Release` (**275**), `Genre` (**2**), `Critic Score` (**8714**), `User Score` (**6804**), `Rating` (**6871**);

- **Некорректные типы данных:**
`Year of Release` - тип `float64` вместо `int64`,
`EU Sales`, `JP Sales`, `User Score` - тип `object` вместо `float64`;

- **Нестандартные наименования столбцов:**
названия начинаются с заглавной буквы, слова разделены пробелами, что не соответствует общепринятому стилю оформления (snake_case).

## Проверка ошибок в данных и их предобработка

### Приведение наименований столбцов к единому стилю

In [6]:
# Приводим названия столбцов к формату snake_case: 
# с помощью методов .lower() (нижний регистр) и .replace() (пробелы заменяем на нижнее подчёркивание).
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [7]:
# Проверяем корректны ли наименования после преобразования.
df.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [8]:
# Cоздаем копию датафрейма до преобразования для возможности проверить сделанные изменения после предобработки.
tmp = df.copy() 
print(f"Изначальный размер датафрейма: {len(tmp)}")

Изначальный размер датафрейма: 16956


### Приведение типов данных к подходящему формату

- Преобразовываем типы данных в более подходящие, для удобства работы с ними.

In [9]:
# Выводим уникальные значения столбцов с числовыми данными для визуальной оценки и выяснения причины некорректных типов данных столбцов.
display(df['eu_sales'].unique())
display(df['jp_sales'].unique())
display(df['user_score'].unique())
display(df['year_of_release'].unique())

array(['28.96', '3.58', '12.76', '10.93', '8.89', '2.26', '9.14', '9.18',
       '6.94', '0.63', '10.95', '7.47', '6.18', '8.03', '4.89', '8.49',
       '9.09', '0.4', '3.75', '9.2', '4.46', '2.71', '3.44', '5.14',
       '5.49', '3.9', '5.35', '3.17', '5.09', '4.24', '5.04', '5.86',
       '3.68', '4.19', '5.73', '3.59', '4.51', '2.55', '4.02', '4.37',
       '6.31', '3.45', '2.81', '2.85', '3.49', '0.01', '3.35', '2.04',
       '3.07', '3.87', '3.0', '4.82', '3.64', '2.15', '3.69', '2.65',
       '2.56', '3.11', '3.14', '1.94', '1.95', '2.47', '2.28', '3.42',
       '3.63', '2.36', '1.71', '1.85', '2.79', '1.24', '6.12', '1.53',
       '3.47', '2.24', '5.01', '2.01', '1.72', '2.07', '6.42', '3.86',
       '0.45', '3.48', '1.89', '5.75', '2.17', '1.37', '2.35', '1.18',
       '2.11', '1.88', '2.83', '2.99', '2.89', '3.27', '2.22', '2.14',
       '1.45', '1.75', '1.04', '1.77', '3.02', '2.75', '2.16', '1.9',
       '2.59', '2.2', '4.3', '0.93', '2.53', '2.52', '1.79', '1.3', '2.6',
   

array(['3.77', '6.81', '3.79', '3.28', '10.22', '4.22', '6.5', '2.93',
       '4.7', '0.28', '1.93', '4.13', '7.2', '3.6', '0.24', '2.53',
       '0.98', '0.41', '3.54', '4.16', '6.04', '4.18', '3.84', '0.06',
       '0.47', '5.38', '5.32', '5.65', '1.87', '0.13', '3.12', '0.36',
       '0.11', '4.35', '0.65', '0.07', '0.08', '0.49', '0.3', '2.66',
       '2.69', '0.48', '0.38', '5.33', '1.91', '3.96', '3.1', '1.1',
       '1.2', '0.14', '2.54', '2.14', '0.81', '2.12', '0.44', '3.15',
       '1.25', '0.04', '0.0', '2.47', '2.23', '1.69', '0.01', '3.0',
       '0.02', '4.39', '1.98', '0.1', '3.81', '0.05', '2.49', '1.58',
       '3.14', '2.73', '0.66', '0.22', '3.63', '1.45', '1.31', '2.43',
       '0.7', '0.35', '1.4', '0.6', '2.26', '1.42', '1.28', '1.39',
       '0.87', '0.17', '0.94', '0.19', '0.21', '1.6', '0.16', '1.03',
       '0.25', '2.06', '1.49', '1.29', '0.09', '2.87', '0.03', '0.78',
       '0.83', '2.33', '2.02', '1.36', '1.81', '1.97', '0.91', '0.99',
       '0.95', '2.0'

array(['8', nan, '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4',
       '8.2', '9', '7.9', '8.1', '8.7', '7.1', '3.4', '5.3', '4.8', '3.2',
       '8.9', '6.4', '7.8', '7.5', '2.6', '7.2', '9.2', '7', '7.3', '4.3',
       '7.6', '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '9.4', '6.8',
       '6.1', '6.7', '5.4', '4', '4.9', '4.5', '9.3', '6.2', '4.2', '6',
       '3.7', '4.1', '5.8', '5.6', '5.5', '4.4', '4.6', '5.9', '3.9',
       '3.1', '2.9', '5.2', '3.3', '4.7', '5.1', '3.5', '2.5', '1.9', '3',
       '2.7', '2.2', '2', '9.5', '2.1', '3.6', '2.8', '1.8', '3.8', '0',
       '1.6', '9.6', '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2',
       '2.3', '0.5', '1.3', '0.2', '0.6', '1.4', '0.9', '1', '9.7'],
      dtype=object)

array([2006., 1985., 2008., 2009., 1996., 1989., 1984., 2005., 1999.,
       2007., 2010., 2013., 2004., 1990., 1988., 2002., 2001., 2011.,
       1998., 2015., 2012., 2014., 1992., 1997., 1993., 1994., 1982.,
       2016., 2003., 1986., 2000.,   nan, 1995., 1991., 1981., 1987.,
       1980., 1983.])

- По полученным данным видно что в столбцах `eu_sales`, `jp_sales` и `user_score` присутствуют строки ('unknown', 'tbd'), которые и являются причиной типа данных `object` у столбцов. Значения NaN, в случае с этими столбцами не влияют на тип данных в негативном ключе, т.к. данные и должны быть типа `float`. 
Строковые значения могли появится в данных либо из-за ручного ввода, либо в результате миграции данных из иных источников с отличающимися стандартами.
- В столбце `year_of_release` присутствуют значения NaN, поэтому при приведении типов столбец был автоматически преобразован в тип `float`, т.к. `int` не поддерживает пропуски.

In [10]:
# Преобразуем типы данных столбцов в более подходящие. С помощью параметра downcast меняем тип данных на более "компактный" для экономии памяти, 
# также преобразуем тип данных в year_of_release в Int32, для той же цели.
df['eu_sales'] = pd.to_numeric(df['eu_sales'], errors='coerce', downcast='float')
df['jp_sales'] = pd.to_numeric(df['jp_sales'], errors='coerce', downcast='float')
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce', downcast='float')
df['year_of_release'] = pd.to_numeric(df['year_of_release'], errors='coerce').astype('Int32')

In [11]:
# Выводим первые строки для визуальной оценки.
df.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.959999,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.580000,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.760000,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.930000,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.890000,10.22,1.00,NaN,NaN,NaN


Данные в столбцах `eu_sales`, `jp_sales`, `user_score`, `genre`, `rating`, `platform`, `year_of_release` были преобразованы, что позволяет корректно с ними работать. 
- Данные столбцов `eu_sales`, `jp_sales`, `user_score` были приведены к типу `float`, поскольку содержат количество продаж и оценки пользователей.
- Данные столбца `year_of_release` приведены к типу `int`, поскольку в нём указаны только годы, без точных дат и времени, приведение к `datetime` здесь не требуется.

### Поиск и обработка пропусков

- В рамках ознакомления с данными посчитаем количество пропуском в абсолютном и относительном значениях для определения дальнейших действий по подготовке данных к анализу.

In [12]:
# Определяем количество пропусков в каждом столбце в абсолютных и относительных значениях.
missing_stats = pd.DataFrame({'Кол-во пропусков': df.isna().sum(),'Доля пропусков': df.isna().mean().round(4)})
missing_stats

,Кол-во пропусков,Доля пропусков
name,2,0.0001
platform,0,0.0000
year_of_release,275,0.0162
genre,2,0.0001
na_sales,0,0.0000
eu_sales,6,0.0004
jp_sales,4,0.0002
other_sales,0,0.0000
critic_score,8714,0.5139
user_score,9268,0.5466


- Столбцы `genre` и `name` (**0.1%**), это одни и те же записи: отсутствует не только название игры, но и жанр, а также оценки (`critic_score`, `user_score, rating`). Так как пропуски затрагивают всего две строки и содержат сразу несколько недостающих значений, их можно удалить без риска потери информации.

- Столбец `year_of_release` содержит **275** пропусков (**1.62%**). Было принято решение удалить строки в которых отсутствует год выхода игры, поскольку нет смысла заменять значения на среднее или медианное, т.к. год, в данном контексте, относится к категориальным данным. Также, нет смысла заменять пропуски на значения-индикаторы, поскольку эти записи не попадут в выборку согласно условиями задания (период с **2000** по **2013**).

- Столбец `jp_sales` (**0.02%**) - **4** игры, большая часть игр выборки продавалась в Японии, поэтому есть основания считать данные о продажах ошибочными, можно заменить на среднее значение.
- Столбец `eu_sales` (**0.04%**) - **6** игр, у данных игр также были продажи в Европе, вполне резонно заменить значения продаж на среднее значение, как в предыдущем примере.

- Столбцы `critic_score` и `user_score` содержат более **50%** пропусков (**51.39%** и **54.66%** соответственно). Из-за такого большого количества пропусков замена средним или медианным значением приведёт к искажению распределения. Поэтому при категоризации по оценкам разумно исключить строки, где эти значения отсутствуют.

- Столбец `rating` содержит **40.52%** пропусков. Поскольку рейтинг ESRB не участвует в ключевых этапах анализа, его можно оставить без изменений. Он носит ознакомительный характер и не влияет на выводы.

In [13]:
# Удаляем записи с отсутствующими названием, жанром и годом выпуска игры.
df = df.dropna(subset=['name', 'genre', 'year_of_release'])

In [14]:
# Создаем функцию для заполнения пропусков в столбцах jp_sales и eu_sales, среднее значение количества 
# продаж игр с пропуском в этих столбцах будет рассчитано на основе игр выпущенных в том же году и на той же игровой платформе.
def fill_mean_sales(row):
    if pd.isna(row['jp_sales']):
        group = df[(df['platform'] == row['platform']) & (df['year_of_release'] == row['year_of_release'])]
        row['jp_sales'] = round(group['jp_sales'].mean(), 2)
    if pd.isna(row['eu_sales']):
        group = df[(df['platform'] == row['platform']) & (df['year_of_release'] == row['year_of_release'])]
        row['eu_sales'] = round(group['eu_sales'].mean(), 2)
    return row

df = df.apply(fill_mean_sales, axis=1)

### Поиск и обработка явных и неявных дубликатов

- В данном блоке анализируем категориальные данные с целью выявления дубликатов, проводим предварительный анализ для первичной оценки данных и определения дальнейших действий по их нормализации.

In [15]:
# Выводим уникальные значения столбцов с категориальными данными для визуальной оценки и поиска дубликатов.
print(f"Столбец 'platform': {list(df['platform'].unique())}\n")
print(f"Столбец 'genre': {list(df['genre'].unique())}\n")
print(f"Столбец 'rating': {list(df['rating'].unique())}")

Столбец 'platform': ['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA', 'PS4', '3DS', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne', 'WiiU', 'GC', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16', '3DO', 'GG', 'PCFX']

Столбец 'genre': ['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc', 'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure', 'Strategy', 'MISC', 'ROLE-PLAYING', 'RACING', 'ACTION', 'SHOOTER', 'FIGHTING', 'SPORTS', 'PLATFORM', 'ADVENTURE', 'SIMULATION', 'PUZZLE', 'STRATEGY']

Столбец 'rating': ['E', nan, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP']


- После визуальной оценки можно сделать вывод что в столбце `genre` присутствуют неявные дубликаты, все наименования жанров продублированы значениями в верхнем регистре. Причиной этому скорее всего является миграция данных из другого источника с отличающимися стандартами. Проблему решит преобразование таких значений в нижний регистр и удаление пробелов с начала и конца значения.
- При первичном анализе, в столбцах `platform` и `rating` дубликаты не обнаружены. Несмотря на отсутствие видимых дубликатов, преобразуем данные в более удобный формат, а именно, `platform` в нижний регистр, а `rating` в верхний, и удалим лишние пробелы с начала и конца значения для корректного подсчета уникальных значений.
- Также, подсчитаем количество записей для каждого уникального значения, для нахождения самых редких из них, поскольку такие значения могут быть неявными дубликатами.

In [16]:
# Выводим количество уникальных значений в каждом столбце.
print(f"Количество уникальных значений в столбце platform: {df['platform'].nunique()}")
print(f"Количество уникальных значений в столбце genre: {df['genre'].nunique()}")
print(f"Количество уникальных значений в столбце rating: {df['rating'].nunique()}")

Количество уникальных значений в столбце platform: 31
Количество уникальных значений в столбце genre: 24
Количество уникальных значений в столбце rating: 8


In [17]:
# В столбцах platform и genre удаляем пробелы с начала и конца названия и приводим в нижний регистр для поиска неявных дубликатов.
# Используем lambda-выражение с методом .apply(), чтобы применить обработку строк сразу ко всем выбранным столбцам датафрейма.
df[['platform', 'genre', 'name']] = df[['platform', 'genre', 'name']].apply(lambda x: x.str.lower().str.strip())
# В столбце rating удаляем пробелы с начала и конца названия и приводим в верхний регистр для поиска неявных дубликатов.
df['rating'] = df['rating'].str.upper().str.strip()

In [18]:
# Выводим количество уникальных значений в каждом столбце.
display(f"Количество уникальных значений в столбце platform: {df['platform'].nunique()}")
display(f"Количество уникальных значений в столбце genre: {df['genre'].nunique()}")
display(f"Количество уникальных значений в столбце rating: {df['rating'].nunique()}")

'Количество уникальных значений в столбце platform: 31'

'Количество уникальных значений в столбце genre: 12'

'Количество уникальных значений в столбце rating: 8'

In [19]:
# Подсчитываем количество записей для каждого уникального значения, 
# чтобы выявить редко встречающиеся варианты, которые могут быть неявными дубликатами.
display(pd.DataFrame(df['platform'].value_counts()))
display(pd.DataFrame(df['genre'].value_counts()))
display(pd.DataFrame(df['rating'].value_counts()))

,count
platform,
ps2,2154
ds,2147
ps3,1330
wii,1305
x360,1249
psp,1212
ps,1208
pc,972
gba,826


,count
genre,
action,3353
sports,2332
misc,1743
role-playing,1499
shooter,1319
adventure,1313
racing,1250
platform,894
simulation,867


,count
rating,
E,3968
T,2948
M,1558
E10+,1414
EC,8
K-A,3
AO,1
RP,1


- В столбце `rating` обнаружен неявный дубликат, значение K-A (устаревшная оценка системы ESRB, ныне - E). 

In [20]:
# Заменяем значение `K-A` на `E` в трех записях.
df['rating'] = df['rating'].replace({'K-A': 'E'})

In [21]:
# Проверяем количество записей для каждой оценки после примененных изменений.
pd.DataFrame(df['rating'].value_counts())

,count
rating,
E,3971
T,2948
M,1558
E10+,1414
EC,8
AO,1
RP,1


- По итогам преобразований видно что в столбце `genre` присутствовали дубли, после приведения значений к нижнему регистру, уникальных значений в этом столбце стало вдвое меньше, среди них дубликатов нет, все значения уникальны.
- В столбце `rating` обнаружен неявный дубликат - значение `K-A` (Kids to Adults). Данный рейтинг является устаревшим, он действовал в системе ESRB с 1994 по 1998 год и в 1998 был заменен на `E` (Everyone). Значения `K-A` были заменены на `E`.
- В столбце `platform` дубликатов не выявлено - значения уникальны и соответствуют реальным игровым платформам.

In [22]:
# Считаем количество явных дубликатов (дублирующихся строк) в датафрейме.
df.duplicated().sum()

np.int64(235)

In [23]:
# Удаляем явные дубликаты.
df = df.drop_duplicates()

In [24]:
# Проверяем сколько было удалено строк датасета.
a, b = len(tmp), len(df)
print(" Было строк в исходном датасете:", a,
      '\n', "Осталось строк в датасете после обработки:", b,
      '\n', "Удалено строк в датасете после обработки:", a-b,
      '\n', "Процент потерь:", round((a-b)/a*100, 2))

 Было строк в исходном датасете: 16956 
 Осталось строк в датасете после обработки: 16444 
 Удалено строк в датасете после обработки: 512 
 Процент потерь: 3.02


- В столбце `genre` было найдено **12** неявных дубликатов, названия жанра были продублированы в верхнем регистре. При помощи метода `lower()` значения были преобразованы в нижний регистр.
- В столбце `rating` был обнаружен неявный дубликат - значение `K-A` (Kids to Adults). Рейтинг является устаревшим, на данный момент такие игры имеют рейтинг `E` (Everyone).
- Во всём датафрейме было обнаружено и удалено **235** явных дубликатов, что составляет **1,4%** от общего количества записей.
- Также, удалены **275** записей  с отсутствующим значением в столбце `year_of_release`, что составляет **1,6%** от общего количества записей, и **2** записи с отсутствующими значениями в стобцах `name` и `genre` - **0,01%** от общего количества записей.
- В итоге: **512** (**3.02%**) записей были удалены, **12** (**0,07%**) записей подверглись преобразованию.

## Фильтрация данных

- Для категоризации игр по оценкам и определения семи лидирующих платформ по выпущенным играм за период создадим новый датафрейм: 
- `df_actual` (игры выпущенные в период с 2000 по 2013);

In [25]:
# Создаем датафрейм df_actual с играми выпущенными в период с 2000 по 2013, для дальнейшей категоризации по оценкам и выявления топ-7 платформ.
# При помощи метода copy() создаем полную копию датафрейма,чтобы безопасно изменять новый датафрейм.
df_actual = df[df['year_of_release'].between(2000, 2013)].copy()

In [26]:
# Получаем сводную информацию по новосозданному датасету.
df_actual.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12781 entries, 0 to 16954
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             12781 non-null  object 
 1   platform         12781 non-null  object 
 2   year_of_release  12781 non-null  int64  
 3   genre            12781 non-null  object 
 4   na_sales         12781 non-null  float64
 5   eu_sales         12781 non-null  float64
 6   jp_sales         12781 non-null  float64
 7   other_sales      12781 non-null  float64
 8   critic_score     7169 non-null   float64
 9   user_score       6483 non-null   float64
 10  rating           8723 non-null   object 
dtypes: float64(6), int64(1), object(4)
memory usage: 1.2+ MB


## Категоризация данных

### Распределение игр по категориям
- Разделяем все игры попавшие в выборку на три категории (низкая, средняя и высокая оценки) отталкиваясь от оценок пользователей и критиков.
- **Пользователи**: низкая (0 - 2.9), средняя (3 - 7.9), высокая (8 - 10).
- **Критики**: низкая (0 - 29), средняя (30 - 79), высокая (80 - 100).

In [27]:
# Проводим категоризацию данных при помощи функции cut(), в параметре right указываем False, чтобы правая граница интервалов не включалась.
# Добавляем столбец user_score_category в датафрейм.
df_actual['user_score_category'] = pd.cut(df_actual['user_score'], 
                                          bins = [0, 3, 8, 10.1], 
                                          labels = ['низкая оценка', 'средняя оценка', 'высокая оценка'], 
                                          right=False)

In [28]:
# Проводим категоризацию данных при помощи функции cut(), в параметре right указываем False, чтобы правая граница интервалов не включалась.
# Добавляем столбец critic_score_category в датафрейм. 
df_actual['critic_score_category'] = pd.cut(df_actual['critic_score'], 
                                            bins = [0, 30, 80, 101], 
                                            labels = ['низкая оценка', 'средняя оценка', 'высокая оценка'], 
                                            right=False)

In [29]:
# Добавляем дополнительную категорию 'нет оценки' в столбец с категориями для отображения количества игр 
# с отсутствующей оценкой от пользователей, либо критиков.
df_actual['user_score_category'] = df_actual['user_score_category'].cat.add_categories(['нет оценки'])
df_actual['user_score_category'] = df_actual['user_score_category'].fillna('нет оценки')

df_actual['critic_score_category'] = df_actual['critic_score_category'].cat.add_categories(['нет оценки'])
df_actual['critic_score_category'] = df_actual['critic_score_category'].fillna('нет оценки')

In [30]:
# Подсчитываем количество игр в каждой категории (оценки пользователей).
user_score_counts = (
    df_actual.groupby('user_score_category', observed=True)['name']
    .count()
    .reset_index()
    .rename(columns={
        'user_score_category': 'Категория оценки пользователей',
        'name': 'Количество игр'
    }))
user_score_counts

,Категория оценки пользователей,Количество игр
0,низкая оценка,116
1,средняя оценка,4081
2,высокая оценка,2286
3,нет оценки,6298


In [31]:
# Подсчитываем количество игр в каждой категории (оценки критиков).
critic_score_counts = (
    df_actual.groupby('critic_score_category', observed=True)['name']
    .count()
    .reset_index()
    .rename(columns={
        'critic_score_category': 'Категория оценки критиков',
        'name': 'Количество игр'
    }))

critic_score_counts

,Категория оценки критиков,Количество игр
0,низкая оценка,55
1,средняя оценка,5422
2,высокая оценка,1692
3,нет оценки,5612


- По полученным данным видно что большая часть игр вышедших в период 2000 - 2013, в основном получили среднюю оценку от пользователей (63%), и от критиков (76%). Высокую оценку получили 35% (оценка пользователей) и 24% (оценка критиков) игр. Лишь 1.8% и 0.8% игр соответственно получили низкие оценки, что позволяет сделать вывод о высоком качестве большинства проектов того времени.

### Топ-7 игровых платформ

- Определяем топ-7 платформ по выпущенным играм за период. Для этого используем датафрейм `df_actual`.

In [32]:
# Выводим топ-7 платформ по количеству выпущенных игр за период 2000-2013.
df_actual.groupby('platform', observed=True)['name'].agg('count').sort_values(ascending=False).head(7)

platform
ps2     2127
ds      2120
wii     1275
psp     1180
x360    1121
ps3     1087
gba      811
Name: name, dtype: int64

- Отталкиваясь от полученных данных можно сделать вывод что за период с 2000 по 2013 по количеству выпущенных игр явно лидировали две платформы, это: `PlayStation 2` (2127) и `Nintendo DS` (2120), в топ-7 также попали: Nintendo Wii (1275), PlayStation Portable (1180), Xbox 360 (1121), PlayStation 3 (1087) и Game Boy Advance (811).

## Итоги проекта

### Вывод

- Анализ показал, что в период с 2000 по 2013 год самыми популярными игровыми платформами были `PlayStation 2` и `Nintendo DS`, на которые выпустили 2127 и 2120 игр соответственно. Далее в списке располагаются платформы: Nintendo Wii (1275), PlayStation Portable (1180), Xbox 360 (1121), PlayStation 3 (1087) и Game Boy Advance (811).
- Также, отталкиваясь от результатов анализа можно сделать вывод что период с 2000 по 2013 было выпущено достаточно много качественных игр, поскольку большая часть проектов получила средние (63% - пользователи, 76% - критики) и высокие оценки (35% - пользователи, 24% - критики), а лишь незначительная часть - низкие (1.8% и 0.8%).

### Проделанная работа

- Основной датафрейм - `df`, получен путем выгрузки данных из файла `new_games.csv`. Перед анализом датафрейм был приведен в более удобный вид,
а именно:
    - преобразованы наименования всех столбцов (->`snake_case`);
    - преобразованы наименования жанров столбца `genre`, для избавления от неявных дубликатов (`upper_case` -> `lower_case`);
    - произведен подсчет количества записей по каждому значению столбцов `genre`, `rating` и `platform` для нахождения неявных дубликатов;
    - 12 строк в столбце `genre` были приведены к `lower_case`;
    - рейтинг `K-A` в столбце `rating`, который является неявным дубликатом был заменен на актуальный рейтинг для игр подпадающих под него - `E`;
    - удалено 512 строк (235 дублирующаяся строк, 275 строк с пропусками в столбце `year_of_release`, и 2 строки с пропусками в столбцах `name` и `genre`);
    - типы данных столбцов `eu_sales`, `jp_sales`, `user_score`, `genre`, `rating`, `platform` и `year_of_release` приведены к наиболее подходящим;
    - пропуски в столбцах `eu_sales` и `jp_sales` заменены на среднее значение.
    
- Данные для анализа периода 2000 - 2013 получены из датафрейма `df_actual`, который является выборкой из основного датафрейма `df`, полученной путем фильтрации данных по столбцу `year_of_release`.